# packages

In [14]:
import vectorbt as vbt
import numpy as np
import pandas as pd
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetStatus
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
import yaml

# get list of stock and crypto symbols

In [2]:
keys = yaml.safe_load(open('..\keys.yaml', 'r'))

In [3]:
trading_client = TradingClient(keys['paper_key'], keys['paper_secret'])

In [4]:
search_params = GetAssetsRequest(asset_class = AssetClass.CRYPTO, status = AssetStatus.ACTIVE)
crypto_assets = trading_client.get_all_assets(search_params)
len(crypto_assets), crypto_assets[0]

(56,
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('6a521dc8-c56b-4096-bb84-66e0db2d155d'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 1.015228426,
     'min_trade_increment': 1e-09,
     'name': 'Tezos / US Dollar',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'XTZ/USD',
     'tradable': True})

In [5]:
df_crypto_assets = pd.DataFrame([dict(asset) for asset in crypto_assets])
df_crypto_assets.head()

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
0,6a521dc8-c56b-4096-bb84-66e0db2d155d,AssetClass.CRYPTO,AssetExchange.CRYPTO,XTZ/USD,Tezos / US Dollar,AssetStatus.ACTIVE,True,False,False,False,True,1.015228,1.000000e-09,NaN,100.0,[]
1,83932f5b-533e-4c69-876a-a120b0ae01c1,AssetClass.CRYPTO,AssetExchange.CRYPTO,XTZ/USDC,Tezos / USD Coin,AssetStatus.ACTIVE,True,False,False,False,True,1.024275,1.000000e-09,NaN,100.0,[]
2,dfb6d1da-427c-42b6-864d-f227af29e2e9,AssetClass.CRYPTO,AssetExchange.CRYPTO,AAVE/USDT,Aave / USD Tether,AssetStatus.ACTIVE,True,False,False,False,True,0.010111,1.000000e-09,0.01,100.0,[]
3,90a83e2d-e574-404b-9344-74130736b71c,AssetClass.CRYPTO,AssetExchange.CRYPTO,MKR/USD,Maker / US Dollar,AssetStatus.ACTIVE,True,False,False,False,True,0.000777,1.000000e-09,0.50,100.0,[]
4,d31cd0b3-852c-435b-a1ea-ace997ce550a,AssetClass.CRYPTO,AssetExchange.CRYPTO,MKR/USDC,Maker / USD Coin,AssetStatus.ACTIVE,True,False,False,False,True,0.000714,1.000000e-09,NaN,100.0,[]


In [6]:
df_crypto_assets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              56 non-null     object 
 1   asset_class                     56 non-null     object 
 2   exchange                        56 non-null     object 
 3   symbol                          56 non-null     object 
 4   name                            56 non-null     object 
 5   status                          56 non-null     object 
 6   tradable                        56 non-null     bool   
 7   marginable                      56 non-null     bool   
 8   shortable                       56 non-null     bool   
 9   easy_to_borrow                  56 non-null     bool   
 10  fractionable                    56 non-null     bool   
 11  min_order_size                  56 non-null     float64
 12  min_trade_increment             56 non

# get list of active stocks

In [7]:
search_params = GetAssetsRequest(asset_class = AssetClass.US_EQUITY,status=AssetStatus.ACTIVE)
stock_assets = trading_client.get_all_assets(search_params)
len(stock_assets), stock_assets[0]

(11069,
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('f8974a61-2f69-4362-9913-09f9810816b1'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'RITE AID CORP COM PAR $1.00',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'RADCQ',
     'tradable': False})

In [8]:
df_stock_assets = pd.DataFrame([dict(asset) for asset in stock_assets])
df_stock_assets.head()

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
0,f8974a61-2f69-4362-9913-09f9810816b1,AssetClass.US_EQUITY,AssetExchange.OTC,RADCQ,RITE AID CORP COM PAR $1.00,AssetStatus.ACTIVE,False,False,False,False,False,None,None,None,100.0,[]
1,55d02f3f-6267-4d57-a747-016a26204dfc,AssetClass.US_EQUITY,AssetExchange.OTC,RBCN,RUBICON TECHNOLOGY INC Common Stock,AssetStatus.ACTIVE,False,False,False,False,False,None,None,None,100.0,[]
2,7c7cbe89-e964-4aa3-bb77-bc0f7f20b617,AssetClass.US_EQUITY,AssetExchange.OTC,LOVLQ,SPARK NETWORKS SE American Depositary Receipts...,AssetStatus.ACTIVE,False,False,False,False,False,None,None,None,100.0,[]
3,01c162ad-1891-4c59-9428-86e2e5922c1f,AssetClass.US_EQUITY,AssetExchange.OTC,ALPSQ,"Alpine Summit Energy Partners, Inc. Class A Su...",AssetStatus.ACTIVE,False,False,False,False,False,None,None,None,100.0,[]
4,4e872f2e-f5ad-417b-9f12-832fb16c03cb,AssetClass.US_EQUITY,AssetExchange.OTC,BRCNF,Burcon NutraScience Corp.,AssetStatus.ACTIVE,False,False,False,False,False,None,None,None,100.0,[]


# get data - all cryptos

In [17]:
client = CryptoHistoricalDataClient()

# Creating request object
request_params = CryptoBarsRequest(
                        symbol_or_symbols=df_crypto_assets['symbol'].tolist(),
                        timeframe=TimeFrame.Day,
                        start=datetime(2000, 9, 1),
                        end=datetime.now()
                        )

In [19]:
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
df_crypto = btc_bars.df
df_crypto

open         high          low  \
symbol    timestamp                                                          
AAVE/USDC 2023-08-18 05:00:00+00:00    55.943866    56.478969    54.992050   
          2023-08-19 05:00:00+00:00    55.947874    57.099034    55.560846   
          2023-08-20 05:00:00+00:00    56.335100    57.579500    56.290500   
          2023-08-21 05:00:00+00:00    57.032000    57.545000    54.970500   
          2023-08-22 05:00:00+00:00    55.075200    55.735350    52.795000   
...                                          ...          ...          ...   
YFI/USD   2023-12-20 06:00:00+00:00  8442.777000  8532.646500  8203.149000   
          2023-12-21 06:00:00+00:00  8273.455000  8476.670000  8254.135500   
          2023-12-22 06:00:00+00:00  8415.272000  8535.400000  8276.495000   
          2023-12-23 06:00:00+00:00  8419.337500  8501.595000  8304.850000   
          2023-12-24 06:00:00+00:00  8387.244000  8460.402000  8310.606500   

                                           close  volume  trade_count  vwap  
symbol    timestamp                                                          
AAVE/USDC 2023-08-18 05:00:00+00:00    55.944857     0.0          0.0   0.0  
          2023-08-19 05:00:00+00:00    56.500850     0.0          0.0   0.0  
          2023-08-20 05:00:00+00:00    56.988250     0.0          0.0   0.0  
          2023-08-21 05:00:00+00:00    55.066550     0.0          0.0   0.0  
          2023-08-22 05:00:00+00:00    55.654250     0.0          0.0   0.0  
...                                          ...     ...          ...   ...  
YFI/USD   2023-12-20 06:00:00+00:00  8271.045000     0.0          0.0   0.0  
          2023-12-21 06:00:00+00:00  8415.417000     0.0          0.0   0.0  
          2023-12-22 06:00:00+00:00  8421.941500     0.0          0.0   0.0  
          2023-12-23 06:00:00+00:00  8379.280000     0.0          0.0   0.0  
          2023-12-24 06:00:00+00:00  8449.305000     0.0          0.0   0.0  

[34733 rows x 7 columns]

In [21]:
#df_crypto.to_pickle('..\\data\\df_crypto_daily.pickle')

In [22]:
df_crypto = pd.read_pickle('..\\data\\df_crypto_daily.pickle')

# df_crypto audit

In [23]:
df_crypto.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 34733 entries, ('AAVE/USDC', Timestamp('2023-08-18 05:00:00+0000', tz='UTC')) to ('YFI/USD', Timestamp('2023-12-24 06:00:00+0000', tz='UTC'))
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         34733 non-null  float64
 1   high         34733 non-null  float64
 2   low          34733 non-null  float64
 3   close        34733 non-null  float64
 4   volume       34733 non-null  float64
 5   trade_count  34733 non-null  float64
 6   vwap         34733 non-null  float64
dtypes: float64(7)
memory usage: 2.0+ MB


In [25]:
df_crypto.index.get_level_values('symbol').unique()

Index(['AAVE/USDC', 'AAVE/USDT', 'AAVE/USD', 'AVAX/USDC', 'AVAX/USDT',
       'AVAX/USD', 'BAT/USDC', 'BAT/USD', 'BCH/BTC', 'BCH/USDC', 'BCH/USDT',
       'BCH/USD', 'BTC/USDC', 'BTC/USDT', 'BTC/USD', 'CRV/USDC', 'CRV/USD',
       'DOGE/USDC', 'DOGE/USDT', 'DOGE/USD', 'DOT/USDC', 'DOT/USD', 'ETH/BTC',
       'ETH/USDC', 'ETH/USDT', 'ETH/USD', 'GRT/USD', 'GRT/USDC', 'LINK/BTC',
       'LINK/USDC', 'LINK/USDT', 'LINK/USD', 'LTC/BTC', 'LTC/USDC', 'LTC/USDT',
       'LTC/USD', 'MKR/USDC', 'MKR/USD', 'SHIB/USDC', 'SHIB/USDT', 'SHIB/USD',
       'SUSHI/USDC', 'SUSHI/USDT', 'SUSHI/USD', 'UNI/BTC', 'UNI/USDC',
       'UNI/USDT', 'UNI/USD', 'USDC/USD', 'USDT/USDC', 'USDT/USD', 'XTZ/USDC',
       'XTZ/USD', 'YFI/USDC', 'YFI/USDT', 'YFI/USD'],
      dtype='object', name='symbol')

In [27]:
df_crypto.index.get_level_values('timestamp').min()

Timestamp('2021-01-01 06:00:00+0000', tz='UTC')

In [38]:
s_btc = df_crypto.loc[('BTC/USD'), 'close']
s_btc

timestamp
2021-01-01 06:00:00+00:00    29676.7900
2021-01-02 06:00:00+00:00    33769.5200
2021-01-03 06:00:00+00:00    32908.0200
2021-01-04 06:00:00+00:00    30441.5700
2021-01-05 06:00:00+00:00    35063.0000
                                ...    
2023-12-20 06:00:00+00:00    43503.5500
2023-12-21 06:00:00+00:00    44164.6955
2023-12-22 06:00:00+00:00    43626.7550
2023-12-23 06:00:00+00:00    43501.8450
2023-12-24 06:00:00+00:00    43684.9250
Name: close, Length: 1088, dtype: float64

# custom indicator

In [40]:
#TODO MA crossover strategy built-in vs. custom indicator


In [42]:
signal = ma_strategy(s_btc)

C:\Users\User\AppData\Local\Temp\ipykernel_11652\3972113582.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if close[x] < mavg*lower_multiplier:
C:\Users\User\AppData\Local\Temp\ipykernel_11652\3972113582.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif close[x] > mavg*upper_multiplier:


In [43]:
pd.Series(signal).describe()

count    358.000000
mean       0.821229
std        0.383697
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64